In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)

In [ ]:
# !unzip /content/drive/MyDrive/SAR/colorization/Input_Sorted-20250323T013151Z-001.zip -d /content/drive/MyDrive/SAR/colorization/

In [ ]:
# !unzip /content/drive/MyDrive/SAR/colorization/Output_Sorted-20250323T013202Z-001.zip -d /content/drive/MyDrive/SAR/colorization/

In [ ]:
color_dir = '/content/drive/MyDrive/SAR/colorization/Output_Sorted'
gray_dir = '/content/drive/MyDrive/SAR/colorization/Input_Sorted'

In [ ]:
print(os.listdir(color_dir)[:5])
print(os.listdir(gray_dir)[:5])

['ROIs1868_summer_s2_59_p11.png', 'ROIs1868_summer_s2_59_p10.png', 'ROIs1868_summer_s2_59_p14.png', 'ROIs1868_summer_s2_59_p15.png', 'ROIs1868_summer_s2_59_p13.png']
['ROIs1868_summer_s1_59_p10.png', 'ROIs1868_summer_s1_59_p12.png', 'ROIs1868_summer_s1_59_p11.png', 'ROIs1868_summer_s1_59_p14.png', 'ROIs1868_summer_s1_59_p13.png']


In [ ]:
SIZE = 256
HEIGHT = SIZE
WIDTH = SIZE
ImagePath = color_dir
N = 500

def ExtractInput(path):
    X_img = []
    y_img = []
    for imageDir in os.listdir(ImagePath)[:N]:
        try:
            img = cv2.imread(os.path.join(ImagePath, imageDir))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            img = img.astype(np.float32)
            img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
            #Convert the rgb values of the input image to the range of 0 to 1
            #1.0/255 indicates that we are using a 24-bit RGB color space.
            #It means that we are using numbers between 0–255 for each color channel
            #img_lab = 1.0/225*img_lab
            # resize the lightness channel to network input size
            img_lab_rs = cv2.resize(img_lab, (WIDTH, HEIGHT)) # resize image to network input size
            img_l = img_lab_rs[:,:,0] # pull out L channel
            #img_l -= 50 # subtract 50 for mean-centering
            img_ab = img_lab_rs[:,:,1:]#Extracting the ab channel
            img_ab = img_ab/128
            #The true color values range between -128 and 128. This is the default interval
            #in the Lab color space. By dividing them by 128, they too fall within the -1 to 1 interval.
            X_img.append(img_l)
            y_img.append(img_ab)
        except:
            pass
    X_img = np.array(X_img)
    y_img = np.array(y_img)

    return X_img,y_img

In [ ]:
X_, y_ = ExtractInput(ImagePath)

In [ ]:
from keras.layers import Conv2D,MaxPooling2D,UpSampling2D,Input,BatchNormalization,LeakyReLU,concatenate
from keras.models import Model

def InstantiateModel(in_):
    model_ = Conv2D(16,(3,3),padding='same',strides=1)(in_)
    model_ = LeakyReLU()(model_)
    #model_ = Conv2D(64,(3,3), activation='relu',strides=1)(model_)
    model_ = Conv2D(32,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)
    model_ = MaxPooling2D(pool_size=(2,2),padding='same')(model_)

    model_ = Conv2D(64,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)
    model_ = MaxPooling2D(pool_size=(2,2),padding='same')(model_)

    model_ = Conv2D(128,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)

    model_ = Conv2D(256,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)

    model_ = UpSampling2D((2, 2))(model_)
    model_ = Conv2D(128,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)

    model_ = UpSampling2D((2, 2))(model_)
    model_ = Conv2D(64,(3,3), padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    #model_ = BatchNormalization()(model_)

    concat_ = concatenate([model_, in_])

    model_ = Conv2D(64,(3,3), padding='same',strides=1)(concat_)
    model_ = LeakyReLU()(model_)
    model_ = BatchNormalization()(model_)

    model_ = Conv2D(32,(3,3),padding='same',strides=1)(model_)
    model_ = LeakyReLU()(model_)
    #model_ = BatchNormalization()(model_)

    model_ = Conv2D(2,(3,3), activation='tanh',padding='same',strides=1)(model_)

    return model_

In [ ]:
Input_Sample = Input(shape=(HEIGHT, WIDTH,1))
Output_ = InstantiateModel(Input_Sample)
Model_Colourization = Model(inputs=Input_Sample, outputs=Output_)

In [ ]:
from keras.optimizers import Adam

LEARNING_RATE = 0.001
Model_Colourization.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                            loss='mean_squared_error')
Model_Colourization.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 16)   │            160 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 256, 256, 16)   │              0 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 32)   │          4,640 │ leaky_re_lu[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU) │ (None, 256, 256, 32)   │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 256, 256, 32)   │            128 │ leaky_re_lu_1[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 32)   │              0 │ batch_normalization[0… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 64)   │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU) │ (None, 128, 128, 64)   │              0 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 128, 128, 64)   │            256 │ leaky_re_lu_2[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 64)     │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 64, 64, 128)    │         73,856 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_3 (LeakyReLU) │ (None, 64, 64, 128)    │              0 │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 64, 64, 128)    │            512 │ leaky_re_lu_3[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_4 (LeakyReLU) │ (None, 64, 64, 256)    │              0 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 64, 64, 256)    │          1,024 │ leaky_re_lu_4[0][0]    │
│ (BatchNormalization) 

 Total params: 820,386 (3.13 MB)

 Trainable params: 819,042 (3.12 MB)

 Non-trainable params: 1,344 (5.25 KB)

In [ ]:
def GenerateInputs(X_,y_):
    for i in range(len(X_)):
        X_input = X_[i].reshape(1,SIZE,SIZE,1)
        y_input = y_[i].reshape(1,SIZE,SIZE,2)
        yield (X_input,y_input)

from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X_,y_, random_state=42)

Model_Colourization.fit(
    GenerateInputs(X_, y_),
    epochs=50,
    verbose=1,
    steps_per_epoch=38,
    validation_data=GenerateInputs(X_val, y_val),
    validation_steps=10
)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 19s 70ms/step - loss: 0.1437 - val_loss: 0.9273
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0100 - val_loss: 0.3687
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0097 - val_loss: 0.3052
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0069 - val_loss: 0.2821
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0061 - val_loss: 0.0245
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 4.8402e-04 - val_loss: 0.0115
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 1.5154e-04 - val_loss: 0.0049
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 2.2625e-04 - val_loss: 0.0037
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 4.0940e-04 - val_loss: 5.8039e-04
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 2.5300e-04 - val_loss: 0.0014
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 6.6181e-04 - val_loss: 2.6262e-04
Epoch 12/50
38/38 ━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.0409e-05 - val_loss: 2.4679e-04
Epoch 16/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.5433e-05 - val_loss: 2.4108e-04
Epoch 17/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.0395e-05 - val_loss: 2.2698e-04
Epoch 18/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5175e-05 - val_loss: 2.2920e-04
Epoch 19/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7634e-05 - val_loss: 2.2230e-04
Epoch 20/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.7529e-05 - val_loss: 2.2110e-04
Epoch 21/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4140e-05 - val_loss: 1.9473e-04
Epoch 22/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4299e-05 - val_loss: 1.9178e-04
Epoch 23/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5109e-05 - val_loss: 1.8559e-04
Epoch 24/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3622e-05 - val_loss: 1.8099e-04
Epoch 25/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2696e-05 - val_loss: 

In [ ]:
Model_Colourization.save('/content/drive/MyDrive/SAR/colorization-CNN.keras')